In [171]:
from scipy.io import loadmat
import numpy as np
import pandas as pd

from pathlib import Path
import cv2
import matplotlib.pylab as plt

cmap = plt.cm.tab20.colors

In [ ]:
p_glob = Path(r'Y:\Neha sapkal\flat chamber- sucrose- 2-17-23-29.34\RE-TRACKED').glob(r'**/*-track.mat')

for p_mat in p_glob:
    p_dir = p_mat.parent.parent
    p_txt = [ p for p in p_dir.glob('*.txt')][0]
    p_png = [ p for p in p_dir.glob('*.png')][0]

    m = loadmat(p_mat, squeeze_me=True, struct_as_record=False)
    data = vars(m['trk'])['data'][:, :, [0, 1]]
    pnt = np.loadtxt(p_txt).astype(int)
    print(p_png)
    img = cv2.imread(str(p_png))
    
    df = pd.DataFrame()
    for i, d in enumerate(data):

        fig, ax = plt.subplots()

        # plot photo
        ax.imshow(img)
        # plot line defining points
        ax.scatter(pnt[:,0], pnt[:,1], zorder=99, color='k')

        # define line separating left and right
        line = dict()
        for p1, p2 in zip(pnt, pnt[1:]):
            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            pxl = np.max([dx, dy])

            x = np.linspace(p1[0], p2[0], pxl + 1).astype(int)
            y = np.linspace(p1[1], p2[1], pxl + 1).astype(int)

            # plot line definition
            ax.scatter(x, y, marker=',', s=1, zorder=98, color='k')

            line = {**line, **{ j: i for i, j in zip(x, y)}}


        # drop nan frames
        fnan = np.isnan(d).any(axis=1)
        d = d[~fnan]

        x = d[:, 0]
        y = d[:, 1]

        # masks for left and right of line
        bl = np.array([line[int(j)] >= i for i, j in zip(x, y)])
        br = np.array([line[int(j)] < i for i, j in zip(x, y)])

        # plot trajectory separated by left and right
        ax.scatter(x[br], y[br], marker=',', s=1, color=cmap[2*i + 1])
        ax.scatter(x[bl], y[bl], marker=',', s=1, color=cmap[2*i])

        fig.savefig(p_dir / 'right_left_fly_{}.png'.format(i+1))

        # calculate velocity

        # store in dataframe
        df.loc[i, 'fly'] = i + 1
        df.loc[i, 'left'] = bl.sum()
        df.loc[i, 'right'] = br.sum()
        df.loc[i, 'left/right'] = bl.sum() / br.sum()
        df.loc[i, 'dropped_frames'] = fnan.sum()


    cols = ['fly', 'left', 'right', 'dropped_frames']
    df.loc[:, cols] = df.loc[:, cols].astype(int)
    df.to_csv(p_dir / 'right_left.csv')
